In [15]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from hurst import compute_Hc
import plotly.express as px
import seaborn as sns

import pandas as pd
import numpy as np
import vectorbt as vbt


import warnings 
warnings.filterwarnings('ignore')


In [81]:
DATA_DIR = 'data/'

In [ ]:
def run_strategy(df, X, slippage):
    df = df.copy()
    
    long_signal = (entry_prices['ret'] > X)
    short_signal = (entry_prices['ret'] < -X)
    
    df['entry_long'] = False
    df['entry_short'] = False
    df['exit'] = False
    
    df.loc[entry_prices[long_signal].index, 'entry_long'] = True
    df.loc[entry_prices[short_signal].index, 'entry_short'] = True
    df.loc[exit_prices.index, 'exit'] = True

    entries = df['entry_long']
    short_entries = df['entry_short']
    exits = df['exit']

    size = 1

    portfolio = vbt.Portfolio.from_signals(
        close=df['price'],
        entries=entries,
        short_entries=short_entries,
        exits=exits,
        short_exits=exits,
        size=size,
        size_type = 'amount',
        slippage = slippage,
        # sl_stop=0.001,
        sl_stop=0.0001,   # stop-loss        
        tp_stop=0.0001,   # take profit
        freq='1T'
    )

    return portfolio


fname = f"{DATA_DIR}AUDUSD_data.parquet"
df = 

df['price'] = (df['bid'] + df['offer']) / 2
df['date'] = df.index.floor('D')

grouped = df.groupby('date')
prev_close = grouped['price'].last().shift(1)


entry_price_time = pd.to_datetime("23:00:00").time()
exit_price_time = pd.to_datetime("23:59:00").time()

entry_mask = df.index.time == entry_price_time
exit_mask = df.index.time == exit_price_time

entry_prices = df[entry_mask].copy()
exit_prices = df[exit_mask].copy()

entry_prices['prev_close'] = entry_prices['date'].map(prev_close)
entry_prices['ret'] = (entry_prices['price'] - entry_prices['prev_close']) / entry_prices['prev_close']

In [76]:
# np.arange(0.0005, 1, 0.001)

In [83]:
import numpy as np
import itertools

# Параметры подбора
X_values = np.arange(0.0005, 0.5, 0.001)    
slippage_values = np.arange(1, 2, 1) / 100

results = []

for X_val, sl_val in tqdm(itertools.product(X_values, slippage_values)):
    pf = run_strategy(df, X=X_val, slippage=sl_val)
    stats = pf.stats()
    results.append({
        'X': X_val,
        'slippage_usd': sl_val,
        'total_return': stats['Total Return [%]'],
        'sharpe': stats['Sharpe Ratio'],
        'drawdown': stats['Max Drawdown [%]']
    })

# В датафрейм для удобного анализа
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='total_return', ascending=False))


500it [01:17,  6.45it/s]

          X  slippage_usd  total_return     sharpe  drawdown
250  0.2505          0.01      0.000000        inf       NaN
329  0.3295          0.01      0.000000        inf       NaN
342  0.3425          0.01      0.000000        inf       NaN
341  0.3415          0.01      0.000000        inf       NaN
340  0.3405          0.01      0.000000        inf       NaN
..      ...           ...           ...        ...       ...
4    0.0045          0.01     -2.170660 -12.076118  2.170735
3    0.0035          0.01     -2.607683 -13.254041  2.607758
2    0.0025          0.01     -3.159537 -14.591787  3.159612
1    0.0015          0.01     -3.855242 -16.130922  3.855317
0    0.0005          0.01     -4.877223 -18.153017  4.877298

[500 rows x 5 columns]


In [80]:
fig_3d = px.scatter_3d(
    results_df,
    x="X",
    y="sharpe",
    z="total_return",
    # color="P-value",
    # size="Correlation_abs",
    title="3D визуализация зависимости корреляции от LookBack и Hold"
)

fig_3d.show()